In [13]:
import numpy as np
import igl
import meshplot as mp

In [14]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

In [15]:
def binning(pi, bbox_min, bbox_max, reso):
    size_of_bin = (bbox_max - bbox_min) / reso
    # print(bbox_max, bbox_min, size_of_bin)

    cube = {}

    for i, p in enumerate(pi):
        # print(cube)
        bin = indexing(p, bbox_min, reso)
        if bin in cube:
            cube[bin].append(i)
        else: 
            cube[bin] = [i]
            
    return cube

def indexing(p, bbox_min, reso):
    x, y, z = (p - bbox_min) / reso
    return (int(x), int(y), int(z))

thing = indexing(np.array([2,2,2]), np.array([1,1,1]), 1)
print(thing)

(1, 1, 1)


# Reading point cloud

In [ ]:
# def find_closest_point(point, points):
#     distance = np.linalg.norm((points-point), axis=0)
#     return np.argmin(distance)

In [17]:
def find_closest_point(point, points):
    distances = np.linalg.norm(points-point, axis = 1)
    return np.argmin(distances)
    
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)

p = []
f = []
bbd = igl.bounding_box_diagonal(pi)

for i, point in enumerate(pi):
    p.append(point)
    f.append(0)
    n = ni[i]

    eps = 0.01 * bbd
    pip = point + eps * n
    while find_closest_point(pip, pi) != i: 
        eps /= 2
        pip = point + eps * n
    p.append(pip)
    f.append(eps)

    eps = 0.01 * bbd
    pim = point - eps * n
    while find_closest_point(pip, pi) != i: 
        eps /= 2
        pip = point - eps * n
    p.append(pim)
    f.append(-eps)

p = np.array(p)
f = np.array(f)
thing = mp.plot(pi, shading={"point_color": "blue", "point_size": 8})
thing.add_points(p[f > 0], shading = {"point_color": "pink", "point_size": 8})
thing.add_points(p[f < 0], shading = {"point_color": "green", "point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

2

# MLS function

In [22]:
# Parameters
bbox_min = np.min(pi, axis=0)
bbox_max = np.max(pi, axis=0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 30

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

In [ ]:
# def closest_points(point, points, h, bbox_min, cube, reso):
#     bin = indexing(point, bbox_min, reso)
#     l = []
#     # if len(l) > 8: print(l)
#     for dx in [-1, 0, 1]:
#         for dy in [-1, 0, 1]:
#             for dz in [-1, 0, 1]:
#                 neigh_bin = (bin[0] + dx, bin[1] + dy, bin[2] + dz)
#                 if neigh_bin in cube:
#                     l += [i for i, x in enumerate(points[cube[neigh_bin]] - point) if np.linalg.norm(x) <= h and i not in l]
#     # if len(l) > 0: print(l)
#     return l


In [26]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

import math

# r is distance between x and ci
# h is wendland radius
def w(x, points, h):
    weights = []
    for p in points:
        r = np.linalg.norm(x-p)
        if r < h:
            weights.append(math.pow(1-(r/h), 4) * (4*r/h + 1))
    return weights

def closest_points(point, points, h):
    # bin = indexing(point, bbox_min, reso)
    return [i for i, x in enumerate(points - point) if np.linalg.norm(x) <= h]
    
    # l = []
    # # if len(l) > 8: print(l)
    # for dx in [-1, 0, 1]:
    #     for dy in [-1, 0, 1]:
    #         for dz in [-1, 0, 1]:
    #             neigh_bin = (bin[0] + dx, bin[1] + dy, bin[2] + dz)
    #             if neigh_bin in cube:
    #                 l += [i for i, x in enumerate(points[cube[neigh_bin]] - point) if np.linalg.norm(x) <= h and i not in l]
    # # if len(l) > 0: print(l)
    # return l

k = {0:1, 1:4, 2:10}
wendlandr = 30
size = p.shape[0]
W = np.zeros((size, size))
fx = np.zeros(len(x))

for i, xi in enumerate(x):
    index = np.array(closest_points(xi, p, wendlandr))
    num_points = len(index)
    if num_points < 2 * k[1]: 
        fx[i] = 7
        continue
        
    # bin = indexing(point, bbox_min, reso)
    W = np.diagflat(w(xi, p, wendlandr))
    B = np.ones((num_points, k[1]))
    B[:, 1:] = p[index]

    # print(B.T.shape, W.shape, f[index].shape)
    A = B.T @ W @ B
    b = B.T @ W @ f[index]
    a = np.linalg.solve(A, b)
    
    # sum = 0
    b = [1,xi[0], xi[1], xi[2]]
    sum_x = b@a
    fx[i] = sum_x
    # break


In [27]:
# Threshold fx to visualize inside outside

color = np.zeros((len(x), 3))
color[fx >= 0] = [1,0,0]
color[fx < 0] = [0,1,0]
mp.plot(x, c=color, shading={"point_size": 6,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Marching to extract surface

In [28]:
# Marching tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
comp = igl.face_components(sf)
num = np.argmax(np.bincount(comp))
new_f = sf[comp == num]
mp.plot(sv, new_f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(4.5712823…